<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
#!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:

from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input


#### Here, I reduced batch size to 80 since my local machine ,which has only 4 GB RAM, could not keep up with 100 batch size. And also, Keras GPU doesn't still support for AMD Radeon 530 GPU. So models were trained on 4GB RAM + 4 vCPU as I wanted to keep my model on my local machine. And I tried training models on IBM Watson Studio 4 vCPU + 16 GB RAM, but for me, it's better off-line training and saving models in local machine. 

In [4]:
num_classes = 2

image_resize = 224

batch_size_training = 80
batch_size_validation = 80

data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [5]:
####Training Set
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')


####Validation Set
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [7]:


model = Sequential()
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

model.layers[0].trainable = False
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 514       
Total params: 14,846,530
Trainable params: 131,842
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2


In [10]:
#Fitting model

fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
376/376 [==============================] - 11235s 30s/step - loss: 0.0268 - accuracy: 0.9907 - val_loss: 0.0057 - val_accuracy: 0.9978
Epoch 2/2
376/376 [==============================] - 11201s 30s/step - loss: 0.0079 - accuracy: 0.9976 - val_loss: 0.0050 - val_accuracy: 0.9985


In [12]:
model.save('classifier_vgg16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [29]:
from keras.models import load_model

resnet50_model = load_model('classifier_resnet50_model.h5')   #This resnet model was trained on my local machine with 80 batch size. .
vgg16_model = load_model('classifier_vgg16_model.h5')  

In [30]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [31]:
data_generator_1 = ImageDataGenerator()

In [32]:
test_generator = data_generator_1.flow_from_directory(
                "concrete_data_week4/test", 
                target_size=(image_resize, image_resize),
                shuffle = False)

Found 500 images belonging to 2 classes.


In [34]:
print("The Performace by ResNet50 is : ")
resnet50 = resnet50_model.evaluate_generator(test_generator)
print(resnet50)
print("Loss : ", str(resnet50[0]))
print("Accuracy : ", str(resnet50[1]))

The Performace by ResNet50 is : 
[0.08974144607782364, 0.9760000109672546]
Loss :  0.08974144607782364
Accuracy :  0.9760000109672546


In [35]:
print("The Performace by VGG16 is : ")
vgg16 = vgg16_model.evaluate_generator(test_generator)
print(vgg16)
print("Loss : ", str(vgg16[0]))
print("Accuracy : ", str(vgg16[1]))

The Performace by VGG16 is : 
[0.04849214479327202, 0.9879999756813049]
Loss :  0.04849214479327202
Accuracy :  0.9879999756813049


### Anyway, VGG16 model has better accuracy than Resnet50. Models were trained with the same specification. 

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [37]:
resnet50_pred = resnet50_model.predict_generator(test_generator)
vgg16_pred = vgg16_model.predict_generator(test_generator)


In [57]:
def pred(x):
    for i in x:
        j = np.argmax(i)
        if(j==0):
            print("Negative")
        else:
            print("Positive")

In [68]:
import numpy as np
print("ResNet50 Class Prediction for first 5 images in testset")
pred(resnet50_pred[0:5])

ResNet50 Class Prediction for first 5 images in testset
Negative
Negative
Negative
Negative
Negative


In [69]:
print("VGG16 Class Prediction for first 5 images in testset")
pred(vgg16_pred[0:5])

VGG16 Class Prediction for first 5 images in testset
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).